### 필요한 라이브러리(프로그래밍 도구) 설치 및 설정

* 필요한 프로그래밍 라이브러리 도구를 설치하고 설정하는 과정입니다.
* 만약 이 코드를 다시 쓸 일이 있다면 반드시 이 부분을 실행해주어야 합니다!!!

In [ ]:
#fastai2 라이브러리 설치
!pip install fastai2

#한글 폰트 처리를 위한 라이브러리 설치
!pip install koreanize-matplotlib

#fastbook 라이브러리 설치 및 불러오기
#실행이 안되면 다시 실행할 것~

!pip install -Uqq fastbook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### CSV 파일 업로드 후 독립변수, 종속변수 설정

* 계속 강조하지만 AI 학습을 위해서는 데이터가 필요하며
* CSV는 테이블(표) 형태의 데이터로 AI 학습에 널리 사용됩니다.
* 첫줄은 데이터 속성(데이터의 이름)이 오고 두번째 줄부터 데이터가 누적됩니다.
* AI 학습이란 그 데이터 중 알고 싶은 내용(종속변수)를 알고 싶은 내용에 관련 있는 요소(독립변수)를 설정한 후 독립변수가 종속변수에 얼마나 영향을 주는지를 데이터를 통해 찾아내는 과정입니다.
* CSV를 업로드 한 후 독립변수, 종속변수 설정 후 왼쪽부터 버튼 2개를 누르고 작업해줘야 합니다!
* 여러분들이 설정한 결과는 CONFIG에 저장됩니다!!

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files
import os
import plotly.express as px
import plotly.io as pio
from PIL import Image
from IPython.display import display
from IPython.core.display import HTML

# 파일 업로드 버튼
upload_button = widgets.Button(description="CSV 파일 업로드", button_style='success')
output = widgets.Output()

# 업로드된 파일 이름과 데이터프레임 저장용 전역 변수
uploaded_file_name = None
df = None  # 전역 데이터프레임

# 경로 설정
path = "/content/drive/MyDrive/AI기초 서비스 수행(수치 회귀)"
if not os.path.exists(path):
    os.makedirs(path)
    print(f"경로가 존재하지 않아 새로 생성했습니다: {path}")
else:
    print(f"파일이 저장될 경로: {path}")

# 버튼 및 상태 메시지: 전역 생성
variable_button = widgets.Button(description="변수 설정 완료", button_style='primary')
save_button = widgets.Button(description="설정 저장", button_style='success')
save_status = widgets.Label(value="")  # 저장 완료 메시지

# 속성 설정 UI 생성 함수
def create_attribute_settings(df):
    settings = []
    for col in df.columns:
        # 역할 선택 (독립변수, 종속변수, 무시)
        role_dropdown = widgets.Dropdown(
            options=['독립변수', '종속변수', '무시'],
            value='무시',
            description=f'{col}:',
            style={'description_width': 'initial'}
        )

        # 데이터 유형 선택 (숫자형, 범주형)
        dtype_dropdown = widgets.Dropdown(
            options=['숫자형', '범주형'],
            value='숫자형',
            description='데이터 유형:',
            style={'description_width': 'initial'},
            disabled=True  # 기본적으로 비활성화
        )

        # 역할 변경 시 데이터 유형 활성화/비활성화
        def toggle_dtype(change):
            dtype_dropdown.disabled = (change['new'] == '무시')

        role_dropdown.observe(toggle_dtype, names='value')

        # 위젯 묶기
        row = widgets.HBox([role_dropdown, dtype_dropdown])
        settings.append((col, role_dropdown, dtype_dropdown, row))
    return settings

# 버튼 동작 정의
def update_dtype_locks(_):
    global settings
    for _, role, dtype, _ in settings:
        dtype.disabled = (role.value == '무시')
    with output:
        print("변수 설정 완료! 데이터 유형을 수정할 수 있습니다.")

def save_config(_):
    global CONFIG
    independent_vars = []
    dependent_var = None
    ignored_vars = []
    data_types = {}

    for col, role, dtype, _ in settings:
        if role.value == '독립변수':
            independent_vars.append(col)
            data_types[col] = dtype.value
        elif role.value == '종속변수':
            dependent_var = col
            data_types[col] = dtype.value
        else:
            ignored_vars.append(col)

    CONFIG = {
        "종속변수": dependent_var,
        "독립변수": independent_vars,
        "무시할 변수": ignored_vars,
        "데이터 유형": data_types
    }

    save_status.value = "설정이 저장되었습니다!"
    with output:
        print("설정이 저장되었습니다!")
        print(CONFIG)

# 버튼 이벤트 연결 (초기 연결)
variable_button.on_click(update_dtype_locks)
save_button.on_click(save_config)

# 파일 업로드 및 처리 함수
def upload_and_process_file(_):
    global uploaded_file_name, df, settings
    output.clear_output()

    # 파일 업로드
    with output:
        print("CSV 파일을 업로드하세요.")
    uploaded = files.upload()
    if not uploaded:
        with output:
            print("파일 업로드에 실패했습니다. 다시 시도해주세요.")
        return

    # 파일 저장 및 데이터프레임 생성
    for file_name in uploaded.keys():
        uploaded_file_name = file_name
        file_path = os.path.join(path, file_name)  # 파일 저장 경로
        with open(file_path, "wb") as f:
            f.write(uploaded[file_name])  # 파일 저장
        print(f"업로드된 파일이 저장되었습니다: {file_path}")

    # 파일 읽기
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
    except Exception:
        try:
            df = pd.read_csv(file_path, encoding='utf-8-sig')
        except Exception:
            try:
                df = pd.read_csv(file_path, encoding='cp949')
            except Exception:
                with output:
                    print("파일을 열 수 없습니다. 파일 인코딩을 확인해주세요.")
                return

    # 데이터 정보 출력
    with output:
        print("\n데이터 정보:")
        display(df.info())
        print("\n데이터 미리보기:")
        display(df.head())
        print("\n데이터 분포:")
        display(df.describe())
        print("\n데이터 상관도")
        num_df = df.select_dtypes(include='number')

        correlation_matrix = num_df.corr()

        # Plotly를 사용한 상관도 시각화
        fig = px.imshow(correlation_matrix,
                        text_auto=True,
                        color_continuous_scale='Viridis',
                        title='업로드 데이터 상관행렬 ')

        fig.write_html("상관도.html")

    # 속성 설정 UI 생성
    settings = create_attribute_settings(df)
    settings_ui = widgets.VBox([s[3] for s in settings])

    # UI 출력
    display(settings_ui)  # 속성 설정 드롭다운 출력
    display(widgets.HBox([variable_button, save_button]))  # 버튼 출력
    display(save_status)  # 저장 상태 메시지 출력

# 파일 업로드 버튼 클릭 이벤트 연결
upload_button.on_click(upload_and_process_file)

# UI 표시
display(widgets.VBox([upload_button, output]))


경로가 존재하지 않아 새로 생성했습니다: /content/drive/MyDrive/AI기초 서비스 수행(수치 회귀)


Saving (실습용) 아파트가 실거래 가공 데이터 (3).csv to (실습용) 아파트가 실거래 가공 데이터 (3).csv
업로드된 파일이 저장되었습니다: /content/drive/MyDrive/AI기초 서비스 수행(수치 회귀)/(실습용) 아파트가 실거래 가공 데이터 (3).csv


Label(value='')

In [ ]:
print(CONFIG)

{'종속변수': '거래금액(만원)', '독립변수': ['단지명', '전용면적(㎡)', '계약연도', '계약월', '층', '건축년도'], '무시할 변수': ['NO', '주소1', '주소2', '평수(전용)', '계약년월', '계약일', '동', '도로명'], '데이터 유형': {'단지명': '범주형', '전용면적(㎡)': '숫자형', '계약연도': '숫자형', '계약월': '숫자형', '거래금액(만원)': '숫자형', '층': '숫자형', '건축년도': '숫자형'}}


### 1. fastai 신경망 회귀 모델

* 신경망(딥러닝)은 일반적으로 머신러닝보다 성능이 좋다고 알려져있지만
* 모든 경우에 신경망 모델이 효과적인것은 아닙니다.
* 여기서는 신경망을 통해 회귀모델을 만들어보고 성능을 확인해보겠습니다.
* 이후 만든 모델을 통해 값을 예측해보겠습니다!

In [ ]:
import pandas as pd
from fastai.tabular.all import *
import pickle
from sklearn.metrics import r2_score

# 1. 무시할 변수 제거
df = df.drop(CONFIG['무시할 변수'], axis=1, errors='ignore')

# 2. 종속변수 및 독립변수 설정
y_col = CONFIG['종속변수']
x_cols = CONFIG['독립변수']

# 3. 데이터 정규화 및 범주형 변수 처리 (fastai가 지원)
cat_names = [col for col, dtype in CONFIG['데이터 유형'].items() if dtype == '범주형']
cont_names = [col for col, dtype in CONFIG['데이터 유형'].items() if dtype == '숫자형' and col != y_col]

# fastai 데이터로더 생성
dls = TabularDataLoaders.from_df(
    df, y_names=y_col, cat_names=cat_names, cont_names=cont_names, procs=[Categorify, Normalize]
)

# 4. 모델 생성 및 학습 (인공신경망 딥러닝 기반!)
learn = tabular_learner(dls, metrics=rmse)
learn.fit_one_cycle(5)  # 5 에포크 학습

# 5. Normalize 및 Categorify 정보 추출
normalize_procs = [proc for proc in dls.train.procs if isinstance(proc, Normalize)][0]
means = {name: normalize_procs.means[name] for name in cont_names}
stds = {name: normalize_procs.stds[name] for name in cont_names}


# 6. 정규화 여부 포함
def get_normalization_info(name):
    return {
        "is_normalized": name in means,
        "mean": means.get(name, None),
        "std": stds.get(name, None)
    }

normalization_info = {name: get_normalization_info(name) for name in cont_names}

categorify_maps = {
    cat: dls.classes[cat] for cat in cat_names if cat in dls.classes
}


# 7. 메타 데이터 저장
meta_data = {
    "model": learn,  # fastai 모델을 직렬화
    "cat_names": cat_names,
    "cont_names": cont_names,
    "categorify_maps": categorify_maps,  # 범주형 변수 인코딩 정보
    "normalize": normalization_info,  # 정규화 정보 포함
    "y_names": [y_col],
    "procs": ["Categorify", "Normalize"]  # 전처리 정보
}

# 8. 모델 및 CONFIG 저장

model_path = path+"/regression_model(fastai).pkl"
#learn.export(model_path)  # fastai 모델 저장
with open(model_path, "wb") as f:
    pickle.dump(meta_data, f)

print(f"딥러닝 기반 회귀 모델이 {model_path}에 저장되었습니다.")

# 9. 검증 데이터셋에 대한 예측값과 실제값 가져오기
preds, targets = learn.get_preds()
r2 = r2_score(targets.numpy(), preds.numpy())
print(f"모델의 R^2 Score는 {r2}입니다. 만약 r2값이 0.5미만이라면 데이터를 보강(더 많은 데이터)하거나 독립변수 종속변수를 다시 설정해보세요!")



#### 값 예측해보기

* 만들어진 신경망 회귀 모델을 통해 값을 예측해보는 코드입니다.
* 모델이 잘 돌아가는지를 테스트 하기 위해 필요합니다.

In [ ]:

# 10. 사용자 입력 인터페이스 생성
cat_inputs = {}
cont_inputs = {}

print("### 사용자 입력 인터페이스 생성 중...")

# 범주형 입력 위젯 생성
for cat in cat_names:
    if cat in df.columns:
        options = df[cat].unique()
        cat_inputs[cat] = widgets.Dropdown(
            options=options,
            description=f"{cat}:",
            style={"description_width": "initial"}
        )
        display(cat_inputs[cat])

# 연속형 입력 위젯 생성
for cont in cont_names:
    cont_inputs[cont] = widgets.FloatText(
        description=f"{cont}:",
        style={"description_width": "initial"}
    )
    display(cont_inputs[cont])

# 예측 버튼 생성
predict_button = widgets.Button(description="Predict")
output = widgets.Output()

def predict_callback(_):
    with output:
        output.clear_output()
        try:
            # 입력 데이터 준비
            input_data = []

            # 범주형 데이터 인코딩
            for cat in cat_names:
                if cat in cat_inputs:
                    category = cat_inputs[cat].value
                    encoded_value = categorify_maps[cat].o2i[category]  # 인코딩된 값 가져오기
                    input_data.append(encoded_value)

            # 연속형 데이터 정규화
            for cont in cont_names:
                if cont in cont_inputs:
                    raw_value = cont_inputs[cont].value
                    mean = normalization_info[cont]["mean"]
                    std = normalization_info[cont]["std"]
                    normalized_value = (raw_value - mean) / std  # 정규화 수행
                    input_data.append(normalized_value)

            # 입력 데이터를 DataLoader로 변환
            test_dl = dls.test_dl(pd.DataFrame([input_data], columns=cat_names + cont_names))

            # 예측 수행
            prediction = learn.get_preds(dl=test_dl)[0][0].item()
            print(f"예측값: {prediction}")
        except Exception as e:
            print(f"에러 발생: {e}")


predict_button.on_click(predict_callback)
display(predict_button, output)


### 2. 랜덤포레스트 머신러닝 모델

* 랜덤포레스트 모델은 타이타닉 데이터에 설명했던 것처럼
* 여러 트리 알고리즘을 조합해 사용하는 모델입니다.
* 파이썬 sklearn(싸이킷런) 라이브러리를 활용해 모델을 만들고
* 이후 만든 모델을 통해 값을 예측해보겠습니다!

In [ ]:
import pandas as pd
from fastai.tabular.all import *
import pickle
from sklearn.metrics import r2_score
import ipywidgets as widgets
from IPython.display import display
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# 데이터 가져오기
train_x, train_y = dls.train.xs, dls.train.ys
valid_x, valid_y = dls.valid.xs, dls.valid.ys


# Scikit-learn 회귀 모델 (Random Forest)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(train_x, train_y.values.ravel())  # 학습

# 검증 데이터 평가
preds = rf_model.predict(valid_x)
rmse = np.sqrt(mean_squared_error(valid_y, preds))
r2 = r2_score(valid_y, preds)
print(f"Random Forest RMSE: {rmse}")
print(f"Random Forest R2: {r2}")

# Normalize 및 Categorify 정보 추출
normalize_procs = [proc for proc in dls.train.procs if isinstance(proc, Normalize)][0]
means = {name: normalize_procs.means[name] for name in cont_names}
stds = {name: normalize_procs.stds[name] for name in cont_names}


#메타 데이터 형식 저장(모델 속성 정보)
meta_data = {
    "model": rf_model,
    "cat_names": [col for col in CONFIG['독립변수'] if CONFIG['데이터 유형'][col] == "범주형"],  # 범주형 독립변수
    "cont_names": [col for col in CONFIG['독립변수'] if CONFIG['데이터 유형'][col] == "숫자형"],  # 숫자형 독립변수
    "y_names": CONFIG["종속변수"],  # 종속변수
    "procs": ["Categorify", "Normalize"],  # 전처리 정보 (필요 시 수정)
}

# 정규화 여부 포함
def get_normalization_info(name):
    return {
        "is_normalized": name in means,
        "mean": means.get(name, None),
        "std": stds.get(name, None)
    }

normalization_info = {name: get_normalization_info(name) for name in cont_names}

# 범주형 변수 매핑 정보 추출
categorify_maps = {
    cat: dls.classes[cat] for cat in cat_names if cat in dls.classes
}

# Random Forest 모델 저장
meta_data = {
    "model": rf_model,
    "cat_names": cat_names,
    "cont_names": cont_names,
    "categorify_maps": categorify_maps,
    "normalize": normalization_info,
    "y_names": [y_col],
    "procs": ["Categorify", "Normalize"]
}


rf_path = path+"/random_forest_model.pkl"
print(rf_path)

with open(rf_path, "wb") as f:
    pickle.dump(meta_data, f)

print("Random Forest 모델이 Fastai 형식으로 저장되었습니다.")

#### 값 예측해보기

* 만들어진 랜덤 포레스트 회귀 모델을 통해 값을 예측해보는 코드입니다.
* 모델이 잘 돌아가는지를 테스트 하기 위해 필요합니다.

In [ ]:
# 사용자 입력 인터페이스 생성
cat_inputs = {}
cont_inputs = {}

print("### 사용자 입력 인터페이스 생성 중...")

# 범주형 입력 위젯 생성
for cat in cat_names:
    if cat in df.columns:
        options = df[cat].unique()
        cat_inputs[cat] = widgets.Dropdown(
            options=options,
            description=f"{cat}:",
            style={"description_width": "initial"}
        )
        display(cat_inputs[cat])

# 연속형 입력 위젯 생성
for cont in cont_names:
    cont_inputs[cont] = widgets.FloatText(
        description=f"{cont}:",
        style={"description_width": "initial"}
    )
    display(cont_inputs[cont])

# 예측 버튼 생성
predict_button = widgets.Button(description="Predict")
output = widgets.Output()

def predict_callback(_):
    with output:
        output.clear_output()
        try:
            # 입력 데이터 준비
            input_data = []

            # 범주형 데이터 인코딩
            for cat in cat_names:
                if cat in cat_inputs:
                    category = cat_inputs[cat].value
                    encoded_value = categorify_maps[cat].o2i[category]  # 인코딩된 값 가져오기
                    input_data.append(encoded_value)

            # 연속형 데이터 정규화
            for cont in cont_names:
                if cont in cont_inputs:
                    raw_value = cont_inputs[cont].value
                    mean = normalization_info[cont]["mean"]
                    std = normalization_info[cont]["std"]
                    normalized_value = (raw_value - mean) / std  # 정규화 수행
                    input_data.append(normalized_value)

            # 예측 수행

            columns = cat_names + cont_names  # 열 이름 설정
            input_df = pd.DataFrame([input_data], columns=columns)  # DataFrame으로 변환
            prediction = rf_model.predict(input_df)[0]
            print(f"예측값: {prediction}")
        except Exception as e:
            print(f"에러 발생: {e}")

predict_button.on_click(predict_callback)
display(predict_button, output)


### 3. Xgboost 모델 만들고 값 예측(회귀)해보기

* Xgboost 모델 모델은 여러 모델을 조합해 사용하는 앙상블 모델로
* 여러 트리 알고리즘을 조합해 사용하는 모델입니다.
* 파이썬  XGBRegressor 라이브러리를 활용해 모델을 만들고
* 이후 만든 모델을 통해 값을 예측해보겠습니다!

In [ ]:
import pandas as pd
from fastai.tabular.all import *
import pickle
from sklearn.metrics import r2_score
import ipywidgets as widgets
from IPython.display import display
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import numpy as np

# XGBoost 모델 생성
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
xgb_model.fit(train_x, train_y.values.ravel())  # 학습

# 검증 데이터 평가
preds = xgb_model.predict(valid_x)
rmse = np.sqrt(mean_squared_error(valid_y, preds))
r2 = r2_score(valid_y, preds)

print(f"XGBoost RMSE: {rmse}")
print(f"XGBoost R2: {r2}")

# Normalize 및 Categorify 정보 추출
normalize_procs = [proc for proc in dls.train.procs if isinstance(proc, Normalize)][0]
means = {name: normalize_procs.means[name] for name in cont_names}
stds = {name: normalize_procs.stds[name] for name in cont_names}

# 정규화 여부 포함
def get_normalization_info(name):
    return {
        "is_normalized": name in means,
        "mean": means.get(name, None),
        "std": stds.get(name, None)
    }

normalization_info = {name: get_normalization_info(name) for name in cont_names}

# 범주형 변수 매핑 정보 추출
categorify_maps = {
    cat: dls.classes[cat] for cat in cat_names if cat in dls.classes
}

# XGBoost 모델 저장
meta_data = {
    "model": xgb_model,
    "cat_names": cat_names,
    "cont_names": cont_names,
    "categorify_maps": categorify_maps,
    "normalize": normalization_info,
    "y_names": [y_col],
    "procs": ["Categorify", "Normalize"]
}

xgb_path = path+"/xgb_model.pkl"
print(rf_path)

with open(xgb_path, "wb") as f:
    pickle.dump(meta_data, f)

print("XGBoost 모델이 Fastai 형식으로 저장되었습니다.")

#### 값 예측해보기

* 만들어진 XGBoost 회귀 모델을 통해 값을 예측해보는 코드입니다.
* 모델이 잘 돌아가는지를 테스트 하기 위해 필요합니다.

In [ ]:
# 사용자 입력 인터페이스 생성
cat_inputs = {}
cont_inputs = {}

print("### 사용자 입력 인터페이스 생성 중...")

# 범주형 입력 위젯 생성
for cat in cat_names:
    if cat in df.columns:
        options = df[cat].unique()
        cat_inputs[cat] = widgets.Dropdown(
            options=options,
            description=f"{cat}:",
            style={"description_width": "initial"}
        )
        display(cat_inputs[cat])

# 연속형 입력 위젯 생성
for cont in cont_names:
    cont_inputs[cont] = widgets.FloatText(
        description=f"{cont}:",
        style={"description_width": "initial"}
    )
    display(cont_inputs[cont])

# 예측 버튼 생성
predict_button = widgets.Button(description="Predict")
output = widgets.Output()

def predict_callback(_):
    with output:
        output.clear_output()
        try:
            # 입력 데이터 준비
            input_data = []

            # 범주형 데이터 인코딩
            for cat in cat_names:
                if cat in cat_inputs:
                    category = cat_inputs[cat].value
                    encoded_value = categorify_maps[cat].o2i[category]  # 인코딩된 값 가져오기
                    input_data.append(encoded_value)

            # 연속형 데이터 정규화
            for cont in cont_names:
                if cont in cont_inputs:
                    raw_value = cont_inputs[cont].value
                    mean = normalization_info[cont]["mean"]
                    std = normalization_info[cont]["std"]
                    normalized_value = (raw_value - mean) / std  # 정규화 수행
                    input_data.append(normalized_value)

            # 예측 수행
            columns = cat_names + cont_names
            input_df = pd.DataFrame([input_data], columns=columns)  # DataFrame 변환
            prediction = xgb_model.predict(input_df)[0]
            print(f"예측값: {prediction}")
        except Exception as e:
            print(f"에러 발생: {e}")

predict_button.on_click(predict_callback)
display(predict_button, output)
